# clone git and initail setup:

In [ ]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import sys
!rm -r ./ut_ml_tools &> /dev/null
!git clone https://github.com/mortezaomidi24/ut_ml_tools.git &> /dev/null
sys.path.append("/kaggle/working/ut_ml_tools")
print("clone done")

In [ ]:
!pip install -r ./ut_ml_tools/requirements.txt &> /dev/null

In [ ]:
from telegram_bot.telegram_functions import (
    start_telegram_bot,
    send_opts_to_telegram,
)
from tools.diagnose_tools import System_information

In [ ]:
##? ------------------------------------------------------------------    
##? telegram bot:
bot_token = 'add your telegram bot api key' #

history = {} # a dict for add logs and info to it
chat_id = "add your telegram chat id" #fitech_1
bot,history = start_telegram_bot(bot_token = bot_token, chat_id=chat_id , history=history)

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

hyper_params = {
    "sequence_length": 28,
    "input_size": 28,
    "hidden_size": 128,
    "num_layers": 2,
    "num_classes": 10,
    "batch_size": 100,
    "num_epochs": 5,
    "learning_rate": 0.01,
}

###? set opts to telegram:
send_opts_to_telegram(bot,hyper_params)

In [ ]:
# MNIST Dataset
train_dataset = dsets.MNIST(
    root="./data/", train=True, transform=transforms.ToTensor(), download=True
)

test_dataset = dsets.MNIST(root="./data/", train=False, transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=hyper_params["batch_size"], shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=hyper_params["batch_size"], shuffle=False
)

bot.send_message(text=f'train_loader size: {len(train_loader)}')
bot.send_message(text=f'test_loader size: {len(test_loader)}')

In [ ]:
# RNN Model (Many-to-One)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial states
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))

        # Forward propagate RNN
        out, _ = self.lstm(x, (h0, c0))

        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
rnn = RNN(
    hyper_params["input_size"],
    hyper_params["hidden_size"],
    hyper_params["num_layers"],
    hyper_params["num_classes"],
)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=hyper_params["learning_rate"])

In [ ]:
bot.send_message(text=str(criterion))
bot.send_message(text=str(optimizer))

In [ ]:
import time

# Train the Model
total_steps = len(train_dataset) // hyper_params["batch_size"]
train_start_epoch_time = int(time.time())
bot.send_message(text=f'total_steps: {total_steps}')
bot.send_message(text='='*30)
bot.send_message(text='start traning:')

step = 0
for epoch in range(hyper_params["num_epochs"]):
    bot.send_message(text='-'*30)
    bot.send_message(text=f'--> epoch: {epoch}')
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(
            images.view(
                -1, hyper_params["sequence_length"], hyper_params["input_size"]
            )
        )
        labels = Variable(labels)

        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = rnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Compute train accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += float((predicted == labels.data).sum())
        step += 1
        
        # print every 50 step
        if (i + 1) % 50 == 0:
            bot.send_message(f"Epoch [{epoch + 1}/{hyper_params['num_epochs']}], Step [{i + 1}/{total_steps}], Loss: {loss.data.item()}, train accuracy: {100 * correct / total}")
            bot.get_new_order_from_telegram_bot(train_start_epoch_time)
              